In [5]:
from data import load, split, ScaleAbsOne
from gpytorch.constraints import Positive
import torch

In [2]:
df = load()
train, dev, test = split(df, "new_dose-response_matrices", inner_fold=1, outer_fold=1)

100%|██████████| 10/10 [00:06<00:00,  1.57it/s]


In [3]:
train_target = train.pop("PercentageGrowth")
dev_target = dev.pop("PercentageGrowth")
test_target = test.pop("PercentageGrowth")
scaler = ScaleAbsOne()
train = scaler.fit_transform(train.values)
dev = scaler.transform(dev.values)
test = scaler.transform(test.values)

In [6]:
sub = torch.tensor(train[:150]).float()
subt = torch.tensor(train_target[:150].values).float()

In [7]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt

In [15]:
import os
smoke_test = ('CI' in os.environ)
training_iter = 2 if smoke_test else 50

# Training data is 100 points in [0,1] inclusive regularly spaced
train_x = torch.linspace(0, 1, 100)
# True function is sin(2*pi*x) with Gaussian noise
train_y = torch.sin(train_x * (2 * math.pi)) + torch.randn(train_x.size()) * math.sqrt(0.04)

# Wrap training, prediction and plotting from the ExactGP-Tutorial into a function,
# so that we do not have to repeat the code later on
def train(model, likelihood, training_iter=training_iter):
    # Use the adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    for i in range(training_iter):
        # Zero gradients from previous iteration
        optimizer.zero_grad()
        # Output from model
        output = model(sub)
        # Calc loss and backprop gradients
        loss = -mll(output, subt)
        print(loss.item())
        loss.backward()
        optimizer.step()


# def predict(model, likelihood, test_x = torch.linspace(0, 1, 51)):
#     model.eval()
#     likelihood.eval()
#     # Make predictions by feeding model through likelihood
#     with torch.no_grad(), gpytorch.settings.fast_pred_var():
#         # Test points are regularly spaced along [0,1]
#         return likelihood(model(test_x))

def plot(observed_pred, test_x=torch.linspace(0, 1, 51)):
    with torch.no_grad():
        # Initialize plot
        f, ax = plt.subplots(1, 1, figsize=(4, 3))

        # Get upper and lower confidence bounds
        lower, upper = observed_pred.confidence_region()
        # Plot training data as black stars
        ax.plot(train_x.numpy(), train_y.numpy(), 'k*')
        # Plot predictive means as blue line
        ax.plot(test_x.numpy(), observed_pred.mean.numpy(), 'b')
        # Shade between the lower and upper confidence bounds
        ax.fill_between(test_x.numpy(), lower.numpy(), upper.numpy(), alpha=0.5)
        ax.set_ylim([-3, 3])
        ax.legend(['Observed Data', 'Mean', 'Confidence'])


In [16]:
# import positivity constraint
from gpytorch.constraints import Positive

class MultiLinearKernel(gpytorch.kernels.Kernel):
    # the sinc kernel is stationary
    is_stationary = False

    # We will register the parameter when initializing the kernel
    def __init__(self, length_prior=None, length_constraint=None, **kwargs):
        super().__init__(**kwargs)

        # register the raw parameter
        self.register_parameter(
            name='raw_length', parameter=torch.nn.Parameter(torch.ones(*self.batch_shape, 1, 1))
        )

        # set the parameter constraint to be positive, when nothing is specified
        if length_constraint is None:
            length_constraint = Positive()

        # register the constraint
        self.register_constraint("raw_length", length_constraint)

        # set the parameter prior, see
        # https://docs.gpytorch.ai/en/latest/module.html#gpytorch.Module.register_prior
        if length_prior is not None:
            self.register_prior(
                "length_prior",
                length_prior,
                lambda m: m.length,
                lambda m, v : m._set_length(v),
            )

    # now set up the 'actual' paramter
    @property
    def length(self):
        # when accessing the parameter, apply the constraint transform
        return self.raw_length_constraint.transform(self.raw_length)

    @length.setter
    def length(self, value):
        return self._set_length(value)

    def _set_length(self, value):
        if not torch.is_tensor(value):
            value = torch.as_tensor(value).to(self.raw_length)
        # when setting the paramater, transform the actual value to a raw one by applying the inverse transform
        self.initialize(raw_length=self.raw_length_constraint.inverse_transform(value))

    def forward(self, x1, x2, **params):
        prod = torch.einsum("nd, md -> nmd", x1, x2)
        frac = (self.length**2 + prod) / (1 + self.length**2)
        return frac.prod(-1)

In [19]:
# Use the simplest form of GP model, exact inference
class MultiLinearGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super().__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = MultiLinearKernel()

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize the new model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = MultiLinearGPModel(sub, subt, likelihood)


# set to training mode and train
model.train()
likelihood.train()
train(model, likelihood)

# Get into evaluation (predictive posterior) mode and predict
model.eval()
likelihood.eval()
# observed_pred = predict(model, likelihood)
# plot results
# plot(observed_pred)

4466.27685546875
4149.9970703125
3863.368408203125
3603.886962890625
3369.152587890625
3156.885986328125
2964.94970703125
2791.355712890625
2634.272216796875
2492.019775390625
2363.072509765625
2246.047607421875
2139.700927734375
2042.9156494140625
1954.690185546875
1874.1331787109375
1800.4478759765625
1732.9251708984375
1670.9365234375
1613.9205322265625
1561.37939453125
1512.86962890625
1467.99755859375
1426.41162109375
1387.79931640625
1351.8817138671875
1318.409423828125
1287.16015625
1257.934326171875
1230.5531005859375
1204.85595703125
1180.6988525390625
1157.95166015625
1136.4976806640625
1116.231201171875
1097.0565185546875
1078.887451171875
1061.6458740234375
1045.2603759765625
1029.66650390625
1014.8057250976562
1000.6240844726562
987.0734252929688
974.1083374023438
961.688720703125
949.7771606445312
938.33935546875
927.3441772460938
916.7628173828125
906.5689697265625


GaussianLikelihood(
  (noise_covar): HomoskedasticNoise(
    (raw_noise_constraint): GreaterThan(1.000E-04)
  )
)